In [4]:
# !pip install yfinance

In [11]:
import yfinance as yf
import pandas as pd
from sqlalchemy import create_engine, text, inspect
from sqlalchemy.exc import IntegrityError
import pymysql
import os
from dotenv import load_dotenv
load_dotenv()
pymysql.install_as_MySQLdb()

In [32]:
def get_db_engine(user, password, host, db_name):
    """SQLAlchemy 데이터베이스 엔진을 생성합니다."""
    try:
        # PyMySQL 드라이버를 사용하여 엔진 생성
        engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{db_name}")
        # 연결 테스트
        with engine.connect() as connection:
            print("MySQL 데이터베이스 연결 성공")
        return engine
    except Exception as e:
        print(f"엔진 생성 중 오류 발생: {e}")
        return None

def upsert_data(engine, table_name, df):
    """
    데이터를 테이블에 삽입하되, 중복된 경우 업데이트합니다 (Upsert).
    이 방법은 대량의 데이터를 효율적으로 처리합니다.
    """
    # 임시 테이블을 사용하여 데이터를 로드
    temp_table_name = f"temp_{table_name}"
    
    with engine.connect() as connection:
        # 트랜잭션 시작
        with connection.begin() as transaction:
            try:
                # 1. 데이터를 임시 테이블에 저장 (기존 테이블이 있다면 덮어쓰기)
                df.to_sql(temp_table_name, connection, if_exists='replace', index=False)

                # 2. 메인 테이블의 칼럼명 가져오기
                cols = ", ".join([f"`{c}`" for c in df.columns])
                
                # *** 여기가 핵심 수정 사항입니다 ***
                # ON DUPLICATE KEY UPDATE 절을 올바른 MySQL 문법으로 수정
                update_cols = ", ".join([f"`{c}` = VALUES(`{c}`)" for c in df.columns if c not in ['stock_code', 'trade_date']])

                # 3. INSERT ... ON DUPLICATE KEY UPDATE 쿼리 실행
                query = text(f"""
                    INSERT INTO `{table_name}` ({cols})
                    SELECT {cols} FROM `{temp_table_name}`
                    ON DUPLICATE KEY UPDATE {update_cols};
                """)
                connection.execute(query)
                
                # 4. 임시 테이블 삭제
                connection.execute(text(f"DROP TABLE `{temp_table_name}`"))
                
                print(f"'{df['stock_code'].iloc[0]}' 종목의 데이터 {len(df)}건 처리 완료")

            except Exception as e:
                transaction.rollback() # 오류 발생 시 롤백
                print(f"데이터 처리 중 오류: {e}")


def main():
    # 1. MySQL 데이터베이스 연결 정보
    db_host = os.environ.get("DB_HOST")
    db_user = os.environ.get("DB_USER")
    db_password = os.environ.get("DB_PASSWORD")
    db_name =  os.environ.get("DB_name")
    table_name = "stock_prices"
    
    # 데이터베이스 엔진 생성
    engine = get_db_engine(db_user, db_password, db_host, db_name)
    if engine is None:
        return

    # stock code불러와 리스트로 만들기
#     global stock_codes_ori 
    stock_codes_ori = pd.read_sql("stock_company_info", con=engine)
#     display(stock_codes_ori)
    
    import numpy as np

    # 1. 조건 목록을 정의합니다.
    conditions = [
        stock_codes_ori['주식종목'] == '코스닥',
        stock_codes_ori['주식종목'] == '유가증권'
    ]

    # 2. 각 조건에 해당하는 결과 목록을 정의합니다. (접미사 .KS 수정)
    choices = [
        stock_codes_ori['종목코드'] + '.KQ',
        stock_codes_ori['종목코드'] + '.KS'
    ]

    # 3. np.select를 사용하여 조건에 맞는 ticker를 생성합니다.
    # 조건에 해당하지 않는 나머지는 NaN으로 처리 후 제거합니다.
    tickers = np.select(conditions, choices, default=np.nan)
    stock_codes = pd.Series(tickers).dropna().tolist()

    
    # 3. 테이블 존재 여부 확인 및 생성
    inspector = inspect(engine)
    if not inspector.has_table(table_name):
        print(f"'{table_name}' 테이블이 존재하지 않아 새로 생성합니다.")
        create_table_query = text(f"""
        CREATE TABLE {table_name} (
            stock_code VARCHAR(10) NOT NULL,
            trade_date DATE NOT NULL,
            open NUMERIC(20, 2),
            high NUMERIC(20, 2),
            low NUMERIC(20, 2),
            close NUMERIC(20, 2),
            adj_close NUMERIC(20, 2),
            volume BIGINT,
            PRIMARY KEY (stock_code, trade_date)
        ) ENGINE=InnoDB;
        """)
        with engine.connect() as connection:
            connection.execute(create_table_query)
            # SQLAlchemy 2.0+ 에서는 commit이 자동
            # connection.commit() 

    # 4. 각 종목 데이터 다운로드 및 DB 저장
    for code in stock_codes:
        print(f"\n'{code}' 종목 데이터 처리 시작...")
        try:
            data = yf.download(code, period="2y", progress=False)

            # ▼▼▼ [수정] 멀티인덱스 컬럼을 일반 컬럼으로 변경 ▼▼▼
            if isinstance(data.columns, pd.MultiIndex):
                data.columns = data.columns.get_level_values(0)
            # ▲▲▲ 여기까지 ▲▲▲

            if data.empty:
                print(f"'{code}' 데이터 다운로드 실패. 건너뜁니다.")
                continue

            # 데이터프레임 가공
            df = data.reset_index() # Date 인덱스를 컬럼으로 변환

            # ========= ▼▼▼ 이 코드를 추가해야 합니다 ▼▼▼ =========
            # yfinance 최신 버전에 맞춰 'Adj Close'가 없으면 'Close' 값으로 생성
            if 'Adj Close' not in df.columns:
                df['Adj Close'] = df['Close']
            # ========= ▲▲▲ 여기까지 ▲▲▲ =========

            df['stock_code'] = code.split('.')[0] # 종목 코드 컬럼 추가
            # 컬럼명 변경 (DB 컬럼명과 일치시키기)
            df = df.rename(columns={
                'Date': 'trade_date',
                'Open': 'open',
                'High': 'high',
                'Low': 'low',
                'Close': 'close',
                'Adj Close': 'adj_close',
                'Volume': 'volume'
            })
            # 필요한 컬럼만 선택
            df = df[['stock_code', 'trade_date', 'open', 'high', 'low', 'close', 'adj_close', 'volume']]

            # 데이터베이스에 Upsert
            upsert_data(engine, table_name, df)

        except Exception as e:
            print(f"'{code}' 종목 처리 중 오류 발생: {e}")

    print("\n모든 작업 완료")


if __name__ == '__main__':
    main()

MySQL 데이터베이스 연결 성공

'331740.KQ' 종목 데이터 처리 시작...


C:\Users\haram4th\AppData\Local\Temp\ipykernel_16992\2068018861.py:118: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(code, period="2y", progress=False)


'331740' 종목의 데이터 1건 처리 완료

'0030R0.KS' 종목 데이터 처리 시작...


C:\Users\haram4th\AppData\Local\Temp\ipykernel_16992\2068018861.py:118: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(code, period="2y", progress=False)


'0030R0' 종목의 데이터 5건 처리 완료

'462310.KQ' 종목 데이터 처리 시작...


...

'000050' 종목의 데이터 486건 처리 완료

'000700.KS' 종목 데이터 처리 시작...


C:\Users\haram4th\AppData\Local\Temp\ipykernel_16992\2068018861.py:118: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(code, period="2y", progress=False)


'000700' 종목의 데이터 486건 처리 완료

'003480.KS' 종목 데이터 처리 시작...


C:\Users\haram4th\AppData\Local\Temp\ipykernel_16992\2068018861.py:118: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(code, period="2y", progress=False)


'003480' 종목의 데이터 486건 처리 완료

모든 작업 완료
